In [2]:
import json
import time

from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from tqdm.notebook import tqdm

In [3]:
base_url = 'https://trustpilot.com'

def get_soup(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')

In [4]:
import re
soup = get_soup(base_url + '/categories')

NoneType = type(None)
data = {}

for category in soup.findAll('div', {'class' : 'subCategory___BRUDy'}):
    name = category.find('h3', {'class' : 'subCategoryHeader___36ykD'}).text
    name = name.strip()
    data[name] = {}
    sub_categories = category.findAll('div', {'class' : 'subCategoryItem___3ksKz'})
    for sub_category in sub_categories:
        sub_category_item = sub_category.find('a', {'class' : re.compile('subtle___1Gtrs')})
        sub_category_name = sub_category_item.text
        sub_category_uri =  sub_category_item['href']
        data[name][sub_category_name] = sub_category_uri

In [5]:
def extract_company_urls_form_page():
    try:
        a_list = driver.find_elements_by_xpath('//a[@class="wrapper___2rOTx"]')
        urls = [a.get_attribute('href') for a in a_list]
        dedup_urls = list(set(urls))
        return dedup_urls
    except:
        print("Exception in extract_company_urls_from_page")
        return []

def go_next_page():
    try:
        button = driver.find_element_by_xpath('//a[@class="paginationLinkNormalize___scOgG paginationLinkNext___1LQ14"]')
        return True, button
    except NoSuchElementException:
        return False, None

In [6]:
import os
import stat

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(executable_path=r'C:\Users\igor-valeriu.railean\Python\projectsforCV\chromedriver.exe', options=options)

timeout = 3

In [88]:
redirected_categories = {
    'Events & Entertainment'    : {'Gambling' : 'gambling_house', 'Gaming' : 'gaming_service_provider'},
    'Home Services'             : {'Craftsman' : 'handyman'},
    'Money & Insurance'         : {'Insurance' : 'insurance_agency'},
    'Shopping & Fashion'        : {'Accessories' : 'fashion_accessories_store'},
    'Travel & Vacation'         : {'Hotels' : 'hotel', 'Travel Agencies' : 'travel_agency'},
    'Vehicles & Transportation' : {'Bicycles': 'bicycle_store'}
}

def get_url_for_redirected_categories(category, sub_category):
    if category in redirected_categories.keys():
        if sub_category in redirected_categories[category]:
            return redirected_categories[category][sub_category]
    return None

In [90]:
company_urls = {}

pbar_category = tqdm(data)

with open(r'C:\Users\igor-valeriu.railean\Python\projectsforCV\log.txt', 'w') as logfile: 
    for category in tqdm(data):
        pbar_category.set_description("Processing category %s" % category)
        pbar_subcategory = tqdm(data[category], leave=False)
        for sub_category in tqdm(data[category], leave=False):
            pbar_subcategory.set_description("Processing sub_category %s" % sub_category)
            company_urls[sub_category] = []
            
            redirected_url = get_url_for_redirected_categories(category, sub_category)
            
            if redirected_url == None:
                url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all"
            else:
                url = base_url + '/categories/' + redirected_url + "?numberofreviews=0&timeperiod=0&status=all"
                
            driver.get(url)
            logfile.write("Processing url: " + url + "\n")
            
            try:
                element_present = EC.presence_of_element_located(
                    (By.CLASS_NAME, 'wrapper___2rOTx'))
            except:
                pass

            WebDriverWait(driver, timeout).until(element_present)

            next_page = True
            c = 1
            while next_page:
                extracted_company_urls = extract_company_urls_form_page()
                company_urls[sub_category] += extracted_company_urls
                next_page, button = go_next_page()
                
                if next_page:
                    c += 1
                    next_url = url + f'&page={c}'
                    logfile.write("Processing next_url: " + next_url + "\n")
                    
                    driver.get(next_url)
                    try: 
                        element_present = EC.presence_of_element_located(
                            (By.CLASS_NAME, 'wrapper___2rOTx'))

                        WebDriverWait(driver, timeout).until(element_present)
                    except:
                        print("Exception element_present 2")
                        pass
        print("Done with category ", category)
        
consolidated_data = []

for category in data:
    for sub_category in data[category]:
        for url in company_urls[sub_category]:
            consolidated_data.append((category, sub_category, url))

df_consolidated_data = pd.DataFrame(consolidated_data, columns=['category', 'sub_category', 'company_url'])

df_consolidated_data.to_csv('consolidate_company_urls.csv', index=False)

Done with category  Animals & Pets


Done with category  Beauty & Well-being


Done with category  Business Services


Done with category  Construction & Manufacturing


Done with category  Education & Training


...

Done with category  Sports


Done with category  Travel & Vacation


Done with category  Utilities


Done with category  Vehicles & Transportation



NameError: name 'f' is not defined

In [92]:
df = pd.read_csv('consolidate_company_urls.csv')
df.shape

(60217, 3)